In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count,sum, concat_ws, month,year,unix_timestamp,when
from pyspark.sql.window import Window

spark = SparkSession \
        .builder \
        .config("spark.jars",
                "/Users/sahilnagpal/Downloads/mysql-connector-java-8.0.29.jar") \
        .master("local[*]") \
        .appName("leetcode-problems") \
        .getOrCreate()

pizza_delivery = spark.read.format("jdbc"). \
    option("url", "jdbc:mysql://localhost:3306/leetcode") \
    .option("driver", "com.mysql.jdbc.Driver") \
    .option("dbtable", "pizza_delivery") \
    .option("user", "root") \
    .option("password", "password") \
    .load()

In [17]:
pizza_delivery\
    .withColumn("period",concat_ws('-',month("order_time"),year("order_time")))\
    .withColumn("order_timestamp", unix_timestamp("order_time")) \
    .withColumn("delivery_timestamp",unix_timestamp("actual_delivery"))\
    .withColumn("time_diff_minutes", 
                   (col("delivery_timestamp") - col("order_timestamp")) / 60)\
    .groupby("period")\
    .agg(((sum(when(col("time_diff_minutes")>30,col("no_of_pizzas")))/sum("no_of_pizzas"))*100).alias("percentage"),
         (sum(when(col("time_diff_minutes")>30,col("no_of_pizzas")))).alias("total_pizzas"))\
    .show()

+-------+------------------+------------+
| period|        percentage|total_pizzas|
+-------+------------------+------------+
| 1-2023| 8.010335917312661|          31|
|12-2023|15.890410958904111|          58|
| 9-2023|18.816067653276956|          89|
|10-2023|13.274336283185843|          60|
| 7-2023|11.466666666666667|          43|
| 8-2023|12.450592885375494|          63|
| 6-2023| 11.11111111111111|          48|
| 4-2023|17.225950782997764|          77|
| 3-2023|14.805825242718445|          61|
|11-2023|19.553072625698324|         105|
| 2-2023| 10.22964509394572|          49|
| 5-2023|14.412416851441243|          65|
+-------+------------------+------------+

